<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_AdvanceDL_Project/blob/main/models/option_model3_all_put.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import io
import os
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive/')

data_path='/content/gdrive/Shareddrives/CMPE297_49_project/dataset/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LeakyReLU
from tensorflow.keras import backend
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv(data_path+'final_put.csv')
put_df_21 = df[['UnderlyingPrice','Strike','Bid','Ask','sigma_21','date_diff','treasury_rate']]
put_df_21=put_df_21.dropna()

In [22]:
put_df_21.head()

,UnderlyingPrice,Strike,Bid,Ask,sigma_21,date_diff,treasury_rate
5043,13.735,5.5,0.0,0.02,0.029875,2,1.43
5044,13.735,6.0,0.0,0.01,0.029875,2,1.43
5045,13.735,6.5,0.0,0.02,0.029875,2,1.43
5046,13.735,7.0,0.0,0.02,0.029875,2,1.43
5047,13.735,7.5,0.0,0.02,0.029875,2,1.43


In [23]:
put_df_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 676678 entries, 5043 to 702375
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   UnderlyingPrice  676678 non-null  float64
 1   Strike           676678 non-null  float64
 2   Bid              676678 non-null  float64
 3   Ask              676678 non-null  float64
 4   sigma_21         676678 non-null  float64
 5   date_diff        676678 non-null  int64  
 6   treasury_rate    676678 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 41.3 MB


In [24]:
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df_21.drop(['Bid', 'Ask'], axis=1),
                                                                        put_df_21[['Bid', 'Ask']], test_size=0.1, random_state=42)

In [25]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 1024
n_epochs = 1500
learning_rate=1e-5

In [26]:
def make_model3(n_units,layers,learning_rate,shape):
    model = Sequential()
    model.add(Dense(n_units, input_dim=shape))
    model.add(LeakyReLU())
    for _ in range(layers - 1):
      model.add(Dense(n_units))
      model.add(LeakyReLU())

    model.add(Dense(2, activation='relu'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

In [27]:
model3=make_model3(n_units,layers,learning_rate,put_X_train.shape[1])

In [28]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 400)               2400      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 400)              

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
output_path= '/content/gdrive/Shareddrives/CMPE297_49_project/models/'
file_name = 'model3_call21_all_best.h5'
checkpoint = ModelCheckpoint(output_path+file_name,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
history = model3.fit(put_X_train, put_y_train, batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.2, callbacks=[TensorBoard(),checkpoint],verbose=1)

Epoch 1/1500
469/476 [============================>.] - ETA: 0s - loss: 133.7289
Epoch 00001: val_loss improved from inf to 130.02760, saving model to /content/gdrive/Shareddrives/CMPE297_49_project/models/model3_call21_all_best.h5
476/476 [==============================] - 2s 5ms/step - loss: 133.7697 - val_loss: 130.0276
Epoch 2/1500
463/476 [============================>.] - ETA: 0s - loss: 133.6590
Epoch 00002: val_loss improved from 130.02760 to 130.00897, saving model to /content/gdrive/Shareddrives/CMPE297_49_project/models/model3_call21_all_best.h5
476/476 [==============================] - 2s 4ms/step - loss: 133.7729 - val_loss: 130.0090
Epoch 3/1500
472/476 [============================>.] - ETA: 0s - loss: 133.7872
Epoch 00003: val_loss improved from 130.00897 to 130.00523, saving model to /content/gdrive/Shareddrives/CMPE297_49_project/models/model3_call21_all_best.h5
476/476 [==============================] - 2s 4ms/step - loss: 133.7722 - val_loss: 130.0052
Epoch 4/1500


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
tensorboard --logdir /content/logs/

In [ ]:
!tensorboard dev upload \
  --logdir /content/logs/ \
  --name " Model3_put_all" \
  --description "n_units = 400,layers = 4,n_batch = 1024,n_epochs = 1500,learning_rate=1e-5" \
  --one_shot

In [ ]:
model3.evaluate(put_X_test, put_y_test, batch_size=256)

In [ ]:
put_y_pred3 = model3.predict(put_X_test)

In [ ]:
for i in range(20,30):
  print("y: ["+ str(put_y_test.iloc[i][0]),str(put_y_test.iloc[i][1])+"],---- ", "pred: "+ str(put_y_pred3[i]))

In [ ]:
model3.save(output_path+'model3_put_all_1024_1500.h5')